In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time  # Importar el módulo time

# Cargar los archivos de Excel
file_paths = {
    40: '/content/drive/MyDrive/Reto bioinspirados/matrices_40.xlsx',
    100: '/content/drive/MyDrive/Reto bioinspirados/matrices_100.xlsx',
    150: '/content/drive/MyDrive/Reto bioinspirados/matrices_150.xlsx',
    200: '/content/drive/MyDrive/Reto bioinspirados/matrices_200.xlsx'
}

# Definir los parámetros del algoritmo de hormigas
alpha = 1.0  # Importancia de la feromona
beta = 5.0   # Importancia de la distancia
rho = 0.5    # Tasa de evaporación de la feromona
Q = 100      # Constante de la cantidad de feromona depositada
num_ants = 10  # Número de hormigas
num_iterations = 100  # Número de iteraciones

# Función para inicializar la matriz de feromonas
def initialize_pheromone_matrix(num_nodes):
    return np.ones((num_nodes, num_nodes))

# Función para calcular la probabilidad de seleccionar el siguiente nodo
def calculate_transition_probabilities(current_node, unvisited_nodes, pheromone_matrix, distance_matrix):
    pheromones = np.power(pheromone_matrix[current_node, unvisited_nodes], alpha)
    heuristic = np.power(1.0 / distance_matrix[current_node, unvisited_nodes], beta)
    probabilities = pheromones * heuristic
    return probabilities / probabilities.sum()

# Función para ejecutar el recorrido de una hormiga
def run_ant(num_nodes, pheromone_matrix, distance_matrix):
    current_node = 0  # Iniciar en el nodo de origen
    visited_nodes = [current_node]
    total_distance = 0

    for _ in range(num_nodes - 1):
        unvisited_nodes = list(set(range(num_nodes)) - set(visited_nodes))
        probabilities = calculate_transition_probabilities(current_node, unvisited_nodes, pheromone_matrix, distance_matrix)
        next_node = np.random.choice(unvisited_nodes, p=probabilities)
        total_distance += distance_matrix[current_node, next_node]
        visited_nodes.append(next_node)
        current_node = next_node

    # Regresar al nodo inicial
    total_distance += distance_matrix[current_node, 0]
    visited_nodes.append(0)

    return visited_nodes, total_distance

# Función para actualizar la matriz de feromonas
def update_pheromone_matrix(pheromone_matrix, ant_paths, ant_distances):
    num_nodes = len(pheromone_matrix)
    pheromone_matrix *= (1 - rho)  # Evaporación
    for path, distance in zip(ant_paths, ant_distances):
        for i in range(num_nodes):
            pheromone_matrix[path[i], path[i+1]] += Q / distance
            pheromone_matrix[path[i+1], path[i]] += Q / distance
    return pheromone_matrix

# Función principal para resolver el problema TSP usando ACO
def solve_tsp_with_aco(distance_matrix, num_ants, num_iterations):
    num_nodes = len(distance_matrix)
    pheromone_matrix = initialize_pheromone_matrix(num_nodes)
    best_path = None
    best_distance = float('inf')

    for _ in range(num_iterations):
        ant_paths = []
        ant_distances = []

        for _ in range(num_ants):
            path, distance = run_ant(num_nodes, pheromone_matrix, distance_matrix)
            ant_paths.append(path)
            ant_distances.append(distance)
            if distance < best_distance:
                best_distance = distance
                best_path = path

        pheromone_matrix = update_pheromone_matrix(pheromone_matrix, ant_paths, ant_distances)

    return best_path, best_distance

# Procesar cada archivo de Excel
for num_nodes, file_path in file_paths.items():
    excel_data = pd.ExcelFile(file_path)
    fitness_scores = []
    total_distances = []
    best_global_path = None
    best_global_distance = float('inf')

    for sheet_name in excel_data.sheet_names:
        # Cargar la red (distancia) desde el Excel
        distance_matrix = pd.read_excel(excel_data, sheet_name=sheet_name, index_col=0).values

        # Medir el tiempo de resolución
        start_time = time.time()  # Iniciar el temporizador
        best_path, best_distance = solve_tsp_with_aco(distance_matrix, num_ants, num_iterations)
        end_time = time.time()  # Detener el temporizador

        # Calcular el tiempo de resolución
        elapsed_time = end_time - start_time

        # Calcular fitness
        fitness = 1 / best_distance
        fitness_scores.append(fitness)
        total_distances.append(best_distance)

        # Guardar la mejor ruta global
        if best_distance < best_global_distance:
            best_global_distance = best_distance
            best_global_path = best_path

    # Calcular fitness ponderada
    weighted_fitness = np.mean(fitness_scores)
    average_distance = np.mean(total_distances)

    # Imprimir resultados
    print(f"Excel con {num_nodes} nodos:")
    print(f"Fitness ponderada: {weighted_fitness:.4f}")
    print(f"Distancia promedio: {average_distance:.2f}")
    print(f"Mejor ruta: {best_global_path} con distancia {best_global_distance:.2f}")
    print(f"Tiempo computacional: {elapsed_time:.2f} segundos")  # Imprimir el tiempo computacional
    print("\n")


Excel con 40 nodos:
Fitness ponderada: 0.0063
Distancia promedio: 163.49
Mejor ruta: [0, 35, 24, 7, 11, 30, 34, 13, 12, 25, 1, 22, 31, 39, 32, 37, 4, 2, 9, 18, 38, 6, 20, 27, 29, 8, 3, 23, 17, 28, 33, 21, 19, 15, 26, 14, 36, 16, 10, 5, 0] con distancia 130.69
Tiempo computacional: 2.37 segundos


Excel con 100 nodos:
Fitness ponderada: 0.0030
Distancia promedio: 339.80
Mejor ruta: [0, 8, 14, 85, 5, 57, 20, 23, 7, 70, 28, 86, 60, 48, 30, 44, 31, 92, 52, 63, 98, 1, 74, 15, 11, 36, 71, 88, 47, 42, 13, 59, 21, 32, 34, 12, 83, 17, 75, 3, 33, 41, 22, 18, 6, 16, 35, 61, 64, 46, 91, 89, 37, 93, 56, 2, 77, 45, 80, 55, 26, 62, 24, 87, 99, 51, 78, 10, 38, 50, 90, 25, 29, 97, 39, 96, 68, 69, 67, 27, 4, 94, 84, 76, 82, 53, 9, 54, 66, 65, 95, 49, 19, 43, 40, 79, 58, 72, 81, 73, 0] con distancia 253.96
Tiempo computacional: 8.26 segundos


Excel con 150 nodos:
Fitness ponderada: 0.0022
Distancia promedio: 476.05
Mejor ruta: [0, 38, 56, 129, 68, 100, 11, 146, 116, 46, 44, 8, 2, 136, 61, 86, 74, 1, 26,